In [2]:
# Cell 1: Install dependencies
!pip install -q peft accelerate
!pip install -q -U bitsandbytes
!pip install -q transformers qwen-vl-utils pillow pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requ

In [3]:
# Cell 2: Import libraries
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoProcessor,
    Qwen2_5_VLForConditionalGeneration,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from qwen_vl_utils import process_vision_info
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

2025-11-30 02:56:32.393537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764471392.777110      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764471392.965752      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
# Cell 3: Setup paths
train_csv_path = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Train/Train.csv"
train_images_dir = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Train/Image"
test_csv_path = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Test.csv"
test_images_dir = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image"

df = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

# # STRATIFIED SAMPLING: 500 from each class
# df = df.groupby('Label', group_keys=False).apply(
#     lambda x: x. sample(n=min(500, len(x)), random_state=42)
# ). reset_index(drop=True)

print(f"Training samples: {len(df)}")
print(f"Test samples: {len(df_test)}")
print("\nClass distribution:")
print(df['Label'].value_counts())

Training samples: 2860
Test samples: 330

Class distribution:
Label
NonPolitical    2007
Political        853
Name: count, dtype: int64


In [ ]:
# Cell 4: Load processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# processor.image_processor.max_pixels = 602112  # ← ADD THIS LINE

# CHANGE: Reduce max resolution for speed
processor.image_processor.max_pixels = 512 * 512  # 200,704 (down from 12M)


print("Processor loaded")
print(f"Tokenizer vocab size: {len(processor.tokenizer)}")
print(f"Min pixels: {processor.image_processor.min_pixels}")
print(f"Max pixels: {processor. image_processor.max_pixels}")

if processor.tokenizer. pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

print(f"Pad token ID: {processor.tokenizer.pad_token_id}")
print(f"EOS token ID: {processor.tokenizer.eos_token_id}")

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

Processor loaded
Tokenizer vocab size: 151665
Min pixels: 3136
Max pixels: 262144
Pad token ID: 151643
EOS token ID: 151645


In [6]:
# Cell 5: Quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Quantization config created")

Quantization config created


In [7]:
# Cell 6: LoRA config
lora_config = LoraConfig(
    r=8, #16
    lora_alpha=16, #32
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.15,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

print("LoRA config created")
print(f"Rank: {lora_config.r}")
print(f"Alpha: {lora_config.lora_alpha}")

LoRA config created
Rank: 8
Alpha: 16


In [8]:
# Cell 7: Load model
print("Loading Qwen2.5-VL-7B-Instruct...")

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

print("Model loaded")

Loading Qwen2.5-VL-7B-Instruct...


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

Model loaded


In [9]:
# Cell 8: Prepare for LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

trainable_params = sum(p. numel() for p in model. parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"Total params: {total_params:,}")

model.print_trainable_parameters()

Trainable params: 23,794,688 (0.50%)
Total params: 4,716,052,480
trainable params: 23,794,688 || all params: 8,315,961,344 || trainable%: 0.2861


In [ ]:
# Cell 9: Data collator
class PoliticalMemeDataCollator:
    def __init__(self, processor, images_dir):
        self.processor = processor
        self.images_dir = images_dir
        
    def __call__(self, examples):
        texts = []
        images = []
        assistant_responses = []
        
        for example in examples:
            image_name = example["Image_name"]
            image_path = os.path.join(self.images_dir, image_name)
            
            try:
                image = Image.open(image_path).convert("RGB")
            except:
                continue
            
            label = example['Label']
            
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": """Analyze this meme and classify it as either Political or NonPolitical. 

                            Political: Content targeting political figures, parties, ideologies, policies, or political events.
                            NonPolitical: Content without political targeting (general humor, social topics, neutral). 

                            Respond with only one word: Political or NonPolitical."""
                        },
                        {"type": "image", "image": image}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [{"type": "text", "text": label}]
                }
            ]
            
            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)
            texts.append(text. strip())
            images.append([image])
            assistant_responses.append(label)
        
        batch = self.processor(text=texts, images=images, return_tensors="pt", padding=True)
        labels = batch["input_ids"].clone()
        
        for i, (input_ids, assistant_response) in enumerate(zip(batch["input_ids"], assistant_responses)):
            assistant_tokens = self.processor.tokenizer(assistant_response, return_tensors="pt")["input_ids"][0]
            
            start_idx = self.find_subsequence(input_ids, assistant_tokens)
            if start_idx is not None:
                end_idx = start_idx + len(assistant_tokens)
                if end_idx < len(input_ids) and input_ids[end_idx] == self.processor.tokenizer.eos_token_id:
                    end_idx += 1
                
                labels[i, :start_idx] = -100
                labels[i, end_idx:] = -100
        
        batch["labels"] = labels
        return batch
    
    def find_subsequence(self, sequence, subsequence):
        seq_len = len(sequence)
        sub_len = len(subsequence)
        
        for i in range(seq_len - sub_len + 1):
            if torch.equal(sequence[i:i+sub_len], subsequence):
                return i
        return None

data_collator = PoliticalMemeDataCollator(processor=processor, images_dir=train_images_dir)
print("Data collator created")

Data collator created


In [11]:
# Cell 10: Convert to HuggingFace Dataset
from datasets import Dataset as HFDataset

train_dataset = HFDataset. from_pandas(df)
print(f"Dataset created: {len(train_dataset)} samples")

Dataset created: 2860 samples


In [ ]:
# Cell 11: Verify collator
print("Verifying data collator...")

test_batch = data_collator([train_dataset[0]])
original_label = df. iloc[0]['Label']

decoded_labels = processor.tokenizer.decode(
    [token for token in test_batch["labels"][0]. tolist() if token != -100],
    skip_special_tokens=True
)

print(f"Original Label: '{original_label}'")
print(f"Decoded Labels: '{decoded_labels. strip()}'")
print(f"Match: {original_label. strip() == decoded_labels.strip()}")

if original_label.strip() == decoded_labels.strip():
    print("Data collator verified!")
else:
    print("Mismatch detected!")

Verifying data collator...
Original Label: 'Political'
Decoded Labels: 'Political'
Match: True
✅ Data collator verified!


In [ ]:
# Cell 12: Training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/qwen2.5vl-political-checkpoints",
    per_device_train_batch_size=1,       
    gradient_accumulation_steps=8,        
    num_train_epochs=1,                  
    learning_rate=1e-4,
    warmup_steps=100,                      
    logging_steps=20,                    
    save_steps=len(df) // 2,             
    save_strategy="steps",
    fp16=False,
    bf16=True,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    report_to="none",
    seed=42,
    weight_decay=0.01,                  
    max_grad_norm=1.0,                 
    save_total_limit=2,                 
)

print("Training arguments configured")
print(f"Effective batch size: {training_args. per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total steps per epoch: {len(df) // training_args. per_device_train_batch_size}")


# # Cell 12: Training arguments
# training_args = TrainingArguments(
#     output_dir="/kaggle/working/qwen2.5vl-political-checkpoints",
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=16,
#     num_train_epochs=1,
#     learning_rate=2e-4,
#     warmup_steps=100,
#     logging_steps=1,                    # ← CHANGED: Log every step
#     logging_first_step=True,            # ← ADDED: Force first step log
#     save_steps=len(train_dataset),
#     save_strategy="steps",
#     fp16=False,
#     bf16=True,
#     dataloader_pin_memory=False,
#     gradient_checkpointing=True,
#     remove_unused_columns=False,
#     report_to="none",
#     seed=42,
#     save_total_limit=2,
# )

# print("Training arguments configured")
# print(f"Effective batch size: {training_args. per_device_train_batch_size * training_args.gradient_accumulation_steps}")
# print(f"Total steps per epoch: {len(train_dataset) // training_args.per_device_train_batch_size}")

Training arguments configured
Effective batch size: 8
Total steps per epoch: 2860


In [14]:
# Cell 13: Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

print("Starting training...")

trainer.train()

print("Training complete!")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,15.201400
40,2.579800
60,0.000200
80,0.000000
100,0.000000
120,0.000000
140,0.000000
160,0.000000
180,0.000000
200,0.000000


Training complete!


In [15]:
# Cell 14: Save model --> save it as dataset
output_dir = "/kaggle/working/qwen2.5vl-political-lora"

model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Model saved to /kaggle/working/qwen2.5vl-political-lora


In [ ]:
# # Cell 15b: Continue inference without reloading (SAME NOTEBOOK)
# # Use this when you just finished training and want to immediately test

# # Just clear trainer to free memory
# del trainer
# torch.cuda.empty_cache()

# Model is already loaded and fine-tuned - just set to eval mode
model.eval()

print("Model ready for inference (no reloading needed)")
print("Continuing with the already-loaded fine-tuned model...")

✅ Model ready for inference (no reloading needed)
Continuing with the already-loaded fine-tuned model...


In [17]:
# # Cell 15: Reload for inference
# del model
# del trainer
# torch.cuda.empty_cache()

# print("Reloading model for inference...")

# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     quantization_config=quantization_config,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )

# from peft import PeftModel
# # adapter_path = "/kaggle/input/qwen2.5vl-political-lora"  # ← Kaggle dataset path
# # model = PeftModel.from_pretrained(model, adapter_path)

# model = PeftModel.from_pretrained(model, output_dir)
# model.eval()

# print("Model loaded for inference")

In [ ]:
# Cell 16: Inference function
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """Analyze this meme and classify it as either Political or NonPolitical. 

                    Political: Content targeting political figures, parties, ideologies, policies, or political events.
                    NonPolitical: Content without political targeting (general humor, social topics, neutral).

                    Respond with only one word: Political or NonPolitical."""
                },
                {"type": "image", "image": image}
            ]
        }
    ]
    
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    image_inputs, video_inputs = process_vision_info(messages)
    
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ). to(model.device)
    
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            pad_token_id=processor.tokenizer.pad_token_id
        )
    
    generated_text = processor.batch_decode(
        generated_ids[:, inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )[0]
    
    pred_text = generated_text.strip(). lower()
    
    if "political" in pred_text and "non" not in pred_text:
        return "Political"
    else:
        return "NonPolitical"

print("Inference function ready")

Inference function ready


In [19]:
# Cell 17: Test on training samples
print("Testing on sample images...")

for idx in range(min(5, len(df))):
    row = df.iloc[idx]
    img_path = os.path.join(train_images_dir, row['Image_name'])
    if os.path.exists(img_path):
        pred = predict_image(img_path)
        print(f"{row['Image_name']:20s} | True: {row['Label']:15s} | Pred: {pred}")

Testing on sample images...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


train0001.jpg        | True: Political       | Pred: Political


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


train0002.jpg        | True: NonPolitical    | Pred: NonPolitical


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


train0003.jpg        | True: NonPolitical    | Pred: NonPolitical


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


train0004.jpg        | True: NonPolitical    | Pred: NonPolitical
train0005.jpg        | True: NonPolitical    | Pred: NonPolitical


In [20]:
# Cell 18: Run predictions on test set
test_preds = []

for _, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Predicting"):
    image_path = os.path.join(test_images_dir, row['Image_name'])
    
    if not os.path.exists(image_path):
        test_preds.append("NonPolitical")
        continue
    
    try:
        pred = predict_image(image_path)
        test_preds.append(pred)
    except Exception as e:
        print(f"Error on {row['Image_name']}: {e}")
        test_preds.append("NonPolitical")
    
    # if len(test_preds) % 50 == 0:
    #     torch.cuda.empty_cache()

print(f"Predictions complete: {len(test_preds)}")

Predicting: 100%|██████████| 330/330 [21:54<00:00,  3.98s/it]

Predictions complete: 330


In [21]:
# Cell 19: Create submission
submission = pd.DataFrame({
    'Image_name': df_test['Image_name'],
    'Label': test_preds
})

# submission.to_csv('submission.csv', index=False)

submission.to_csv('/kaggle/working/submission.csv', index=False)

print("Submission created!")
print(f"\nPrediction distribution:")
print(submission['Label'].value_counts())
print(f"\nFirst 10 predictions:")
print(submission.head(10))
print("\n" + "="*80)
print("DONE!  Download 'submission.csv' from Output tab")
print("="*80)

Submission created!

Prediction distribution:
Label
NonPolitical    195
Political       135
Name: count, dtype: int64

First 10 predictions:
     Image_name         Label
0  test0001.jpg     Political
1  test0002.jpg  NonPolitical
2  test0003.jpg     Political
3  test0004.jpg  NonPolitical
4  test0005.jpg  NonPolitical
5  test0006.jpg     Political
6  test0007.jpg  NonPolitical
7  test0008.jpg     Political
8  test0009.jpg  NonPolitical
9  test0010.jpg  NonPolitical

DONE!  Download 'submission.csv' from Output tab
